## Graph Construction from Data:

In [ ]:
'''
First few data lines from csv:
Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,14/02/2018 08:31:01,112641719,3,0,0,0,0,0,0,0,0,0,0,0,0,0.0266331163,56320859.5,139.3000358938,56320958,56320761,112641719,56320859.5,139.3000358938,56320958,56320761,0,0,0,0,0,0,0,0,0,0,0,0.0266331163,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,0,0,0,0,56320859.5,139.3000358938,56320958,56320761,Benign
0,0,14/02/2018 08:33:50,112641466,3,0,0,0,0,0,0,0,0,0,0,0,0,0.0266331761,56320733,114.5512985522,56320814,56320652,112641466,56320733,114.5512985522,56320814,56320652,0,0,0,0,0,0,0,0,0,0,0,0.0266331761,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,0,0,0,0,56320733,114.5512985522,56320814,56320652,Benign
0,0,14/02/2018 08:36:39,112638623,3,0,0,0,0,0,0,0,0,0,0,0,0,0.0266338483,56319311.5,301.9345955667,56319525,56319098,112638623,56319311.5,301.9345955667,56319525,56319098,0,0,0,0,0,0,0,0,0,0,0,0.0266338483,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,0,0,0,0,56319311.5,301.9345955667,56319525,56319098,Benign
22,6,14/02/2018 08:40:13,6453966,15,10,1239,2273,744,0,82.6,196.7412368715,976,0,227.3,371.6778922072,544.1615279659,3.8735871865,268915.25,247443.778966007,673900,22,6453966,460997.571428571,123109.423587757,673900,229740,5637902,626433.555555556,455082.21422401,1167293,554,0,0,0,0,488,328,2.3241523119,1.5494348746,0,976,135.0769230769,277.8347599674,77192.1538461539,0,0,0,1,0,0,0,0,0,140.48,82.6,227.3,0,0,0,0,0,0,15,1239,10,2273,65535,233,6,32,0,0,0,0,0,0,0,0,Benign
22,6,14/02/2018 08:40:23,8804066,14,11,1143,2209,744,0,81.6428571429,203.7455453568,976,0,200.8181818182,362.2498635422,380.7331748762,2.839597068,366836.083333333,511356.609732762,1928102,21,8804066,677235.846153846,532416.970958985,1928102,246924,7715481,771548.1,755543.082716951,2174893,90,0,0,0,0,456,360,1.5901743581,1.2494227099,0,976,128.9230769231,279.7630315931,78267.3538461539,0,0,0,1,0,0,0,0,0,134.08,81.6428571429,200.8181818182,0,0,0,0,0,0,14,1143,11,2209,5808,233,6,32,0,0,0,0,0,0,0,0,Benign
22,6,14/02/2018 08:40:31,6989341,16,12,1239,2273,744,0,77.4375,190.8311535538,976,0,189.4166666667,347.6425694023,502.4794183028,4.0061001459,258864.481481481,291724.14791076,951098,20,6989341,465956.066666667,244363.896416351,951098,265831,5980598,543690.727272727,460713.519752371,1254338,78,0,0,0,0,332,252,2.2892000834,1.7169000625,0,976,121.1034482759,265.7086676402,70601.0960591133,0,0,0,1,0,0,0,0,0,125.4285714286,77.4375,189.4166666667,0,0,0,0,0,0,16,1239,12,2273,5808,234,7,20,0,0,0,0,0,0,0,0,Benign
'''

import pandas as pd

# Function to Load data:
def load_data(file_path):
    """
    Load the dataset from a CSV file.
    :param file_path: Path to the CSV file
    :return: DataFrame containing the dataset
    """
    df = pd.read_csv(file_path, low_memory=False)
    return df

filepath = 'Data/02-14-2018.csv'
df = load_data(filepath)
display(df.head())

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,14/02/2018 08:31:01,112641719,3,0,0,0,0,0,...,0,0.0,0.0,0,0,56320859.5,139.300036,56320958,56320761,Benign
1,0,0,14/02/2018 08:33:50,112641466,3,0,0,0,0,0,...,0,0.0,0.0,0,0,56320733.0,114.551299,56320814,56320652,Benign
2,0,0,14/02/2018 08:36:39,112638623,3,0,0,0,0,0,...,0,0.0,0.0,0,0,56319311.5,301.934596,56319525,56319098,Benign
3,22,6,14/02/2018 08:40:13,6453966,15,10,1239,2273,744,0,...,32,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
4,22,6,14/02/2018 08:40:23,8804066,14,11,1143,2209,744,0,...,32,0.0,0.0,0,0,0.0,0.000000,0,0,Benign


In [2]:
df.columns

Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
      

In [ ]:
# Create network graph from the data